# Part 1
Each heavenly body orbits exactly one other thing, so let's use a child/parent relationship to represent this data.

We'll build a dictionary that maps each object to the object it orbits, aka we map each child to its parent. Let's call that dictionary `parents`.

In [46]:
with open('day_06.txt') as f:
    parents = {}
    for line in f.readlines():
        parent, child = line.strip().split(')')
        parents[child] = parent

Our goal is to get the total distances of all objects to the Center of Mass, aka `COM`. For this, let's create a dictionary that maps each object to its total distance to `COM`. We'll call this dictionary `com_distances`.

In [63]:
com_distances = {}

Now let's make a function that can compute the total distance from a single object to the Center of Mass `COM`. This function will be called `get_distance`. 

We'll make this function recursive so we can trace each `child` back by looking at its parent, then its parent's parent, and all the way till we hit `COM`.

Since we'll have to eventually call this function on each planet to get its total distance, we're going to end up retracing a lot of steps along the way. To prevent ourselves from having to recompute anything, we can memoize the `com_distances` as we go. 

To do this, we'll pass the `com_distances` object along through the recursion, updating it as we go and using it to short-circuit our recursion whenever possible.

### For example: 
* Suppose I start with Earth, and compute that it is 1 step away from the Sun, and then compute that the Sun is 1 step away from the black hole at the center of the galaxy (`COM`). 
* That means Earth's distance to `COM` is 2.
* Then let's say next I look at the Moon. I compute that it's 1 step away from Earth, and then since I already know how far the Earth is from the `COM`, I can stop!
* The Moon's distance from the `COM` will just be Earth's distance from `COM` (2) plus Moon's distance from Earth (1).
* So the Moon's distance to `COM` will be 1 + 2 = 3.

In [64]:
def get_distance(parents, com_distances, child):
    if child in com_distances:
        pass
    elif parents[child] == 'COM':
        com_distances[child] = 1
    else:
        com_distances[child] = 1 + get_distance(parents, com_distances, parents[child])
    return com_distances[child]

Now we can iterate through all the planets (`children`) in our data and compute the `com_distances` for each one (if it hasn't already been computed in a previous step!)

In [65]:
for child in parents:
    if child not in com_distances:
        com_distances[child] = get_distance(parents, com_distances, child)

The total of all these distances is our answer for Part 1!

In [81]:
total_distances = sum(com_distances.values())
total_distances

160040

# Part 2
Now we need to figure out how many steps it takes to get from the thing you're orbiting around to the thing Santa is orbiting around.

To do this we're going to step toward the `COM` from `YOU`, step toward the `COM` from `SAN`, and find out where those paths meet.

First, let's just enumerate the full path from `YOU` to the `COM` and keep track of how far we've traveled at each step. We'll store this in a dictionary called `you_distances`.

In [88]:
you_distances = {}
child = 'YOU'
distance = 0
while child in parents:
    you_distances[child] = distance
    child = parents[child]
    distance += 1

Next, let's trace through Santa's planetary lineage, counting how far we've traveled as we go.

As soon as we encounter a planet that is part of the path from `YOU` to `COM`, then we're done! We've found the shortest connection between `SAN` and `YOU`.

In [89]:
child = 'SAN'
distance = 0
while child not in you_distances:
    child = parents[child]
    distance  += 1

The previous loop will halt when the variable `child` has reached the spot where your path and santa's path first meet.

Therefore the distance between `SAN` and `YOU` will just be the distance from `SAN` to `child` plus the distance from `YOU` to `child`. 

Lastly we subtract 2 because we're only supposed to count the steps between your and Santa's parent planets. Whatever!

In [92]:
distance_apart = distance + you_distances[child] - 2
distance_apart

373